In [ ]:
#  error ---df= df.write.format("delta").mode("overwrite").save("Files/Silver_movie") need= mergeschema 

#df.write.format("delta").option("mergeSchema", "true").mode("overwrite").save("Files/Silver/silver_movie_delta_new")
# This will auto-update the schema while keeping existing data.



# Already run this code in silver_layer_notebook

## Gold Transformation

In [1]:
%%sql
select * from gold_movie_new;  

---- show the data in gold_movie_new table

StatementMeta(, ffaa4040-655a-43f1-801d-65349780da3a, 2, Finished, Available, Finished)

<Spark SQL result set with 148 rows and 8 fields>

In [2]:
%%sql

---  Top 10 Highest-Rated Movies
select movie_id,rating, movie_title, genres, avg_ratings
from gold_movie_new
order by avg_ratings desc
limit 10;



 ---AVG_rating ==> Gives the average rating per movie, which is the correct way to measure movie popularity instead of rating.


StatementMeta(, ffaa4040-655a-43f1-801d-65349780da3a, 3, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 5 fields>

In [5]:
%%sql
select distinct release_year from gold_movie_new;   
--- give distinct release year

StatementMeta(, 39692910-6b60-4673-877d-e24bdc985240, 6, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 1 fields>

In [4]:
%%sql
--- Identify Rating Trends Over the Years
select release_year, avg_ratings,count(*) as total_reviews
from  gold_movie_new
GROUP by release_year,avg_ratings
order by release_year; 



StatementMeta(, ffaa4040-655a-43f1-801d-65349780da3a, 5, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 3 fields>

Counts the Number of Movies per Rating:

COUNT(*) AS count → Counts how many times each rating appears.

Groups Data by Rating:

GROUP BY rating → Groups all records by their rating value.

Sorts Ratings in Descending Order:

ORDER BY rating DESC → Shows the highest ratings first.

In [5]:
%%sql
--- ## Ratings Distribution
select rating , count(*) as count
from gold_movie_new
group by rating 
order by rating DESC;

StatementMeta(, ffaa4040-655a-43f1-801d-65349780da3a, 6, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 2 fields>

Counts the Number of Movies Released per Year:

COUNT(*) AS total_movie → Counts how many movies were released in each year.

Groups Data by release_year and avg_ratings:

GROUP BY release_year, avg_ratings → Groups movies by year of release and their average rating.

Sorts Results by Release Year:

ORDER BY release_year → Ensures results are displayed chronologically from oldest to newest.

In [11]:
%%sql
---- ##  Yearly Trends in Movie Ratings

select release_year, count(*) as total_movie , avg_ratings
from gold_movie_new
GROUP BY release_year,avg_ratings
ORDER BY release_year;

StatementMeta(, ffaa4040-655a-43f1-801d-65349780da3a, 13, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 3 fields>

In [32]:
from pyspark.sql.functions import lower, col

df_distinct = spark.sql("SELECT DISTINCT(movie_title) FROM gold_movie_new")
df_lower = df_distinct.withColumn("movie_title", lower(col("movie_title")))

df_lower.show()

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 34, Finished, Available, Finished)

+-------------+
|  movie_title|
+-------------+
|the godfather|
|   the matrix|
| finding nemo|
+-------------+



# Building a Recommendation System For movie Data

In [1]:
%%pyspark


from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 3, Finished, Available, Finished)

These libraries are used for building and evaluating recommendation systems using Apache Spark's MLlib (Machine Learning Library). Here's what each one does:

1. SparkSession (from pyspark.sql)
🔹 Creates a connection to Spark for handling large-scale data processing.


2. ALS (from pyspark.ml.recommendation)
🔹 ALS (Alternating Least Squares) is a collaborative filtering algorithm used for recommendation systems.


Used for predicting user preferences based on past interactions (e.g., movie ratings).

Works well with sparse datasets (like user-item matrices).




3. RegressionEvaluator (from pyspark.ml.evaluation)
🔹 Used to measure the accuracy of the recommendation model.


Evaluates how well the predicted ratings match the actual ratings.

Uses metrics like RMSE (Root Mean Squared Error).

In [2]:
# Initialize Spark session
#spark = SparkSession.builder.appName("MovieRecommendationSystem").getOrCreate()
spark = SparkSession.builder.appName("MovieRecommendationSystem").config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog").getOrCreate()


StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 4, Finished, Available, Finished)

Your Spark session initialization includes Delta Lake configurations, which enable ACID transactions, schema enforcement, and time travel for large-scale data processing. Here's a breakdown of what each part does:





1️⃣ SparkSession.builder.appName("MovieRecommendationSystem")
🔹 Creates a new Spark session named "MovieRecommendationSystem" to process and analyze movie recommendation data.








2️⃣ .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
🔹 Why?

Adds Delta Lake support to Spark.

Enables Delta tables, which support ACID transactions, versioning, and schema enforcement.

📌 Example of Delta Lake Benefits:

If you're storing movie rating data in a Delta table, Delta ensures no corrupted/incomplete data is inserted.















3️⃣ .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
🔹 Why?

Makes Delta Lake the default table format instead of traditional Parquet or CSV.

Allows you to use SQL commands like SELECT * FROM my_delta_table without specifying the Delta format explicitly.


In [40]:
# Load Delta table
delta_path = "abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/movie_analysis_lakehouse.Lakehouse/Files/Gold/movie_gold_delta_new"  # Update this path as needed
df = spark.read.format("delta").load(delta_path)

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 42, Finished, Available, Finished)

In [41]:
df.show(5)

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 43, Finished, Available, Finished)

+------------+--------+-------+------+----------------+-------------+-----------+-----------------+
|release_year|movie_id|user_id|rating|       timestamp|  movie_title|     genres|      avg_ratings|
+------------+--------+-------+------+----------------+-------------+-----------+-----------------+
|        1972|    5019|    179|     3|24-03-2025 10:00|The Godfather|Crime|Drama|3.269230769230769|
|        1972|    5019|    124|     5|24-03-2025 10:02|The Godfather|Crime|Drama|3.269230769230769|
|        1972|    5019|    172|     2|24-03-2025 10:04|The Godfather|Crime|Drama|3.269230769230769|
|        1972|    5019|    175|     1|24-03-2025 11:05|The Godfather|Crime|Drama|3.269230769230769|
|        1972|    5019|    145|     3|24-03-2025 12:06|The Godfather|Crime|Drama|3.269230769230769|
+------------+--------+-------+------+----------------+-------------+-----------+-----------------+
only showing top 5 rows



In [16]:
df.printSchema()

StatementMeta(, ffaa4040-655a-43f1-801d-65349780da3a, 18, Finished, Available, Finished)

root
 |-- release_year: integer (nullable = true)
 |-- movie_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- avg_ratings: double (nullable = true)



In [ ]:
# Select required columns and cast to integer
# df = df.selectExpr("cast(user_id as int) user_id", "cast(movie_id as int) movie_id", "cast(rating as float) rating")


# ------------- no need , because its already in the interger type

### For Continuous Data

In [4]:

# Split data into training and test sets (80-20 split)
(train, test) = df.randomSplit([0.8, 0.2], seed=42)


# Here, we randomly split the dataset into 80 20 manner.


StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 6, Finished, Available, Finished)

In [5]:
# Train ALS model --- aLs Uses matrix factorization to predict unknown ratings.
als = ALS(userCol="user_id", itemCol="movie_id", ratingCol="rating", coldStartStrategy="drop")



# Trains the ALS model on the train dataset




StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 7, Finished, Available, Finished)

🔹 userCol="user_id" → The column representing users.


🔹 itemCol="movie_id" → The column representing movies (items).


🔹 ratingCol="rating" → The column containing ratings given by users.


🔹 coldStartStrategy="drop" → Removes items that do not have sufficient interactions (helps avoid errors during predictions).



In [42]:
model = als.fit(train)

#The ALS algorithm factorizes the user-item matrix into two smaller matrices (user and item features).
#It learns latent factors to predict missing ratings.

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 44, Finished, Available, Finished)

In [ ]:
## need to tune the data , because of Rmse is to high 

als = ALS(
    userCol="user_id", 
    itemCol="movie_id", 
    ratingCol="rating", 
    maxIter=15,       # Increase iterations
    regParam=0.05,    # Adjust regularization
    rank=10,          # Number of latent factors
    coldStartStrategy="drop" #Some users/movies might not have enough ratings.  Set coldStartStrategy="drop"
)
model = als.fit(train)

In [43]:
# Make predictions
predictions = model.transform(test)


# After training the ALS model, we use model.transform(test) to generate movie recommendations for users.

#model.transform(test) takes the test dataset and predicts ratings for user-movie pairs.

#It returns a new DataFrame with an additional prediction column.

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 45, Finished, Available, Finished)

After making predictions, we need to evaluate how well our ALS (Alternating Least Squares) recommendation model performs.

In [45]:
# Evaluate RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Model RMSE:", rmse)


StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 47, Finished, Available, Finished)

Model RMSE: 2.1339462781229006


The RMSE (Root Mean Squared Error) of 2.13 indicates the average error between the predicted and actual ratings in your movie recommendation system.


RMSE measures how far the predictions are from the actual ratings.

Lower RMSE = Better accuracy (closer to real values).

An RMSE of 2.13 means that, on average, your model’s predictions deviate by about 2.13 rating points from actual ratings.



RMSE closer to 1 is better,
so we use hyperparameter tunning


###### Already done


als = ALS(
    userCol="user_id", 
    itemCol="movie_id", 
    ratingCol="rating", 
    maxIter=15,       # Increase iterations
    regParam=0.05,    # Adjust regularization
    rank=10,          # Number of latent factors
    coldStartStrategy="drop" #Some users/movies might not have enough ratings.  Set coldStartStrategy="drop"
)
model = als.fit(train)

In [46]:
# Function to get movie recommendations
def get_recommendations(user_id, model, df, n=3):
    unique_movies = df.select("movie_id").distinct()
    user_movies = unique_movies.withColumn("user_id", lit(user_id))
    predictions = model.transform(user_movies).orderBy("prediction", ascending=False)
    return predictions.limit(n).select("movie_id").toPandas()   # print top 2 recommended movies

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 48, Finished, Available, Finished)

This function get_recommendations(user_id, model, df, n=5) generates top movie recommendations for a given user using an ALS (Alternating Least Squares) recommendation model in PySpark.







unique_movies = df.select("movie_id").distinct()
Extracts distinct movie IDs from the dataset df.

This ensures that recommendations are made only for existing movies.

2 Create a new DataFrame for the user

python
Copy
Edit
user_movies = unique_movies.withColumn("user_id", lit(user_id))
Adds a new column user_id, setting its value to the input user_id.

This step simulates that the user has not rated any of these movies but is considering them.

Predict ratings for all movies

predictions = model.transform(user_movies).orderBy("prediction", ascending=False)
Uses the ALS model (model) to predict ratings for the given user.

Orders the results by the highest predicted rating (descending order).

return predictions.limit(n).select("movie_id").toPandas()
Takes the top n recommended movies.

Selects only the movie_id column.

Converts the result from a PySpark DataFrame to a Pandas DataFrame for easier handling.

In [47]:
from pyspark.sql.functions import lit


StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 49, Finished, Available, Finished)

In [48]:

# Example: Get top 3 recommendations for user 179
recommendations = get_recommendations(179, model, df)
print(recommendations)

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 50, Finished, Available, Finished)

   movie_id
0      5002
1      5019
2      5013


This code retrieves the top 3 movie recommendations for user 179 using a trained ALS (Alternating Least Squares) recommendation model.




Uses user ID = 179 to get personalized movie suggestions.









#### for dealing with categorical data

✅ PySpark → Handles big data (movie dataset).
✅ TfidfVectorizer → Converts movie descriptions into numerical features.
✅ cosine_similarity → Finds movies similar to user preferences.
✅ pandas → Converts Spark results into readable DataFrames.

In [49]:
from pyspark.sql.functions import col
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 51, Finished, Available, Finished)

 from pyspark.sql.functions import col
col is a function in PySpark used to refer to DataFrame columns.

Helps in filtering, selecting, and transforming columns.

2️⃣ from sklearn.feature_extraction.text import TfidfVectorizer
TF-IDF (Term Frequency-Inverse Document Frequency) Vectorizer

Converts text data into numerical vectors based on word importance.

Used in text similarity and recommendation systems.

3️⃣ from sklearn.metrics.pairwise import cosine_similarity
Measures how similar two vectors (text, images, user preferences) are.

Outputs values between 0 (completely different) and 1 (identical).

4️⃣ import pandas as pd
Pandas is a popular Python library for handling tabular data.

Used for data manipulation, cleaning, and analysis.

In [50]:

# Convert Spark DataFrame to Pandas
df_pandas = df.select("movie_title").toPandas()

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 52, Finished, Available, Finished)

1️⃣ Extracts only the "movie_title" column from the Spark DataFrame (df) using .select("movie_title").
2️⃣ Converts the Spark DataFrame to a Pandas DataFrame using .toPandas().

.toPandas()?
PySpark DataFrames are distributed and optimized for big data.

Pandas DataFrames work in-memory and are useful for small-scale processing.

If your dataset is not too large, converting it to Pandas allows you to:

Perform easy operations using Pandas functions.

Use machine learning models from scikit-learn.

In [51]:
# Step 1: Convert movie titles to numerical form
vectorizer = TfidfVectorizer(stop_words='english')
title_matrix = vectorizer.fit_transform(df_pandas["movie_title"])

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 53, Finished, Available, Finished)

TfidfVectorizer converts text into numerical vectors using TF-IDF (Term Frequency-Inverse Document Frequency).

stop_words='english' removes common English words like "the", "is", "in" to focus on important words.

fit_transform() learns the vocabulary from movie titles and converts them into a sparse matrix of numbers.

title_matrix is a numerical representation of all movie titles.

 TF-IDF?

 
Converts text data into a format that can be used for machine learning & similarity calculations.

Reduces the impact of common words while keeping meaningful words important.

Helps in finding similar movie titles using cosine similarity.

In [52]:
# Step 2: Compute similarity
similarity_matrix = cosine_similarity(title_matrix) # for categorical data

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 54, Finished, Available, Finished)

Computes the Cosine Similarity between all movie titles.

cosine_similarity(title_matrix) calculates how similar each movie title is to every other movie title.

The output similarity_matrix is a square matrix where:

Rows & columns represent movies.

Each value (i, j) represents the similarity between movie i and movie j.

Why Use Cosine Similarity?


Measures the angle between two vectors rather than their magnitude.

Works well with TF-IDF, where each movie title is represented as a numerical vector.

Values range from 0 to 1:

1 → Exact match (same movie title).

0 → Completely different titles.



In [53]:
print(df_pandas["movie_title"].head(10))  # Print first 10 movie titles


StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 55, Finished, Available, Finished)

0    The Godfather
1    The Godfather
2    The Godfather
3    The Godfather
4    The Godfather
5    The Godfather
6    The Godfather
7    The Godfather
8    The Godfather
9    The Godfather
Name: movie_title, dtype: object


In [54]:
title = "The Godfather"  # Example movie title
title = title.lower()  # Convert input to lowercase


StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 56, Finished, Available, Finished)

In [55]:
def recommend_movies(title, n=2):
    title = title.lower()  # Convert input to lowercase
    df_pandas["movie_title"] = df_pandas["movie_title"].str.lower()  # Convert dataset to lowercase

    if title not in df_pandas["movie_title"].values:
        print("Movie title not found!")
        return []

    idx = df_pandas[df_pandas["movie_title"] == title].index[0]  # Find index of the movie
    similar_movies = list(enumerate(similarity_matrix[idx]))  # Get similarity scores
    sorted_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)[1:n+1]  # Sort and get top N
    return [df_pandas.iloc[i[0]]["movie_title"] for i in sorted_movies]




StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 57, Finished, Available, Finished)

Defines a function recommend_movies that takes:

title → The movie you want recommendations for.

n → The number of similar movies to return (default is 2).



Converts both user input (title) and dataset movie titles (df_pandas["movie_title"]) to lowercase.

This ensures case-insensitive matching (e.g., "Avatar" and "avatar" are treated the same).

Checks if the input movie exists in the dataset.

If not found, it prints "Movie title not found!" and returns an empty list.

idx = df_pandas[df_pandas["movie_title"] == title].index[0]
Finds the index of the input movie in df_pandas.

similar_movies = list(enumerate(similarity_matrix[idx]))
Finds similarity scores for the input movie using similarity_matrix.

similarity_matrix[idx] → Gives a list of similarity values between the movie at idx and all other movies.

enumerate(similarity_matrix[idx]) → Pairs each similarity score with its index.

In [56]:
%%sql
select distinct(movie_title) from gold_movie_new;

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 58, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 1 fields>

In [57]:
# Example: Recommend movies similar to "the godfather"
print(recommend_movies("The Godfather", n=2))

StatementMeta(, b218528e-763e-48d1-98d6-01a460261624, 59, Finished, Available, Finished)

['the godfather', 'the godfather']


If "The Godfather" is found in the dataset, the function will return two most similar movies based on TF-IDF cosine similarity of titles.